In [51]:
import glob
import os

file = "UCEC"
gene_rank=["TP53","PTEN","JAK1","MTOR","POLE","ATM"]

In [ ]:
import os
import pandas as pd
import numpy as np
"""
This code is used to separate features from different clusters
"""
datalist = os.listdir("./Gene_Mut/"+file+"/TCGA/cluster/all")
for m in datalist:
    out = pd.read_csv("./Gene_Mut/"+file+"/TCGA/cluster/all/"+str(m))
    for i in [1,2,3,4]:
        dif_cluster=[]
        out_clu=out[out["cluster"]==i]
        out_clu=np.array(out_clu)
        dif_cluster=out_clu[:,0:256]
        dif_cluster = dif_cluster.astype(np.float32)
        save_folder_path="./Gene_Mut/"+file+"/TCGA/cluster/cluster_"+str(i)
        if not os.path.exists(save_folder_path):
            os.mkdir(save_folder_path)
        np.save(save_folder_path+"/"+str(m)[0:12]+".npy",dif_cluster)
    print(m)

In [ ]:
import numpy as np
import glob
import os
import concurrent.futures
from functools import partial

"""
This code is used to sample data
"""

def sample_data(data_path, save_folder_path, sample_maximum_number, fill_mode=0, random_seed=1):

    np.random.seed(random_seed)
    data = np.load(data_path)

    shape = data.shape
    numpy_data_type = 0  # 0 mean probability, 1 square numpy array
    data_save_type = ".npy"

    if len(shape) > 2:
        side_length = shape[1]
        feature_dim = shape[3]
        sample_data = np.zeros((sample_maximum_number, side_length, side_length, feature_dim), dtype=np.float32)
        numpy_data_type = 1
    else:  # mean probability
        feature_dim = shape[1]
        sample_data = np.zeros((sample_maximum_number, feature_dim), dtype=np.float32)
        # numpy_data_type = 0
    #
    if fill_mode == 0:  # fill zero

        if shape[0] < sample_maximum_number:
            if numpy_data_type == 1:
                for i in range(shape[0]):
                    sample_data[i, :, :, :] = data[i, :, :, :]
            if numpy_data_type == 0:
                for i in range(shape[0]):
                    sample_data[i, :] = data[i, :]
        else:
            if numpy_data_type == 1:
                sample_list = np.random.choice((data.shape[0]), sample_maximum_number, replace=False)
                sample_data = data[sample_list, :]
            if numpy_data_type == 0:
                sample_list = np.random.choice((data.shape[0]), sample_maximum_number, replace=False)
                sample_data = data[sample_list, :]
    else:
        if shape[0] < sample_maximum_number:
            if numpy_data_type == 1:
                sample_list = np.random.choice((data.shape[0]), sample_maximum_number, replace=True)
                sample_data = data[sample_list, :]
            if numpy_data_type == 0:
                sample_list = np.random.choice((data.shape[0]), sample_maximum_number, replace=True)
                sample_data = data[sample_list, :]
        else:
            if numpy_data_type == 1:
                sample_list = np.random.choice((data.shape[0]), sample_maximum_number, replace=False)
                sample_data = data[sample_list, :]
            if numpy_data_type == 0:
                sample_list = np.random.choice((data.shape[0]), sample_maximum_number, replace=False)
                sample_data = data[sample_list, :]

    midname = data_path[data_path.rindex("/") + 1:data_path.rindex(".")]
    save_path = save_folder_path + "/" + midname + data_save_type
    sample_data = sample_data.astype(np.float32)
    np.save(save_path, sample_data)

if __name__ == '__main__':

    for c in [1,2,3,4]:
        feature_folder_path = "./Gene_Mut/"+file+"/TCGA/cluster/cluster_"+str(c) # "/data/MCO_data/whole_segmentation_36"
        paths = glob.glob(feature_folder_path + "/*")

        sample_maximum_number = 2048
        fill_mode = 1
        save_folder_path = "./Gene_Mut/"+file+"/TCGA/cluster/2048_cluster_"+str(c)  # "/data/MCO_data/sample_data_whole_segmentation_36_mode_" + str(fill_mode)
        random_seed = 12345

        if not os.path.exists(save_folder_path):
            os.mkdir(save_folder_path)

        # sample_data(data_path, save_folder_path, sample_maximum_number, fill_mode=0, random_seed=1)
        """

        with concurrent.futures.ProcessPoolExecutor() as executor:

            executor.map(partial(sample_data, save_folder_path=save_folder_path, sample_maximum_number=sample_maximum_number, fill_mode=fill_mode, random_seed=random_seed), paths)


        """

        for path in paths:
            sample_data(path, save_folder_path=save_folder_path, sample_maximum_number=sample_maximum_number, fill_mode=fill_mode, random_seed=random_seed)

